In [82]:
from transformers import AutoTokenizer
import os
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings

In [83]:
file_path = "./proxmox_documents/PVE_Trouble_Shooting_Paged.pdf"
embedding_model = "BAAI/bge-base-zh-v1.5"

In [84]:
loader = PyMuPDFLoader(file_path)
documents = loader.load()
documents

[Document(metadata={'source': './proxmox_documents/PVE_Trouble_Shooting_Paged.pdf', 'file_path': './proxmox_documents/PVE_Trouble_Shooting_Paged.pdf', 'page': 0, 'total_pages': 10, 'format': 'PDF 1.5', 'title': '', 'author': 'python-docx', 'subject': '', 'keywords': '', 'creator': 'Microsoft® Word 2016', 'producer': 'Microsoft® Word 2016', 'creationDate': "D:20241202104224+08'00'", 'modDate': "D:20241202104224+08'00'", 'trapped': ''}, page_content='PVE 常見問題與 Trouble Shooting \n問題 1：無法啟動虛擬機器 (VM) \n問題類型：VM 啟動問題 \n問題描述：在 Web 介面中嘗試啟動虛擬機器時出現錯誤訊息，如 "Cannot access storage" \n或 "Unable to start VM"。 \n排解方式： \n1. 使用 REST API 檢查虛擬機器狀態：GET \n/api2/json/nodes/{node}/qemu/{vmid}/status/current \n2. 確認儲存設備是否正常運作：GET /api2/json/nodes/{node}/storage \n3. 若儲存設備異常，重新掛載或修復存儲。 \n4. 嘗試手動啟動 VM：POST /api2/json/nodes/{node}/qemu/{vmid}/status/start \n注意事項：請確認 VM 所需的資源可用，並檢查是否有防火牆或權限\n問題。 \n問題 2：無法建立快照 \n問題類型：快照問題 \n問題描述：在進行快照操作時，出現 "Snapshot creation failed" 錯誤。 \n排解方式： \n1. 使用 REST API 確認 VM 狀態是否支援快照：GET \n

In [85]:
def count_tokens_per_page_transformers(documents, model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    token_counts = {}
    for doc in documents:
        page_num = doc.metadata['page']
        token_count = len(tokenizer.encode(doc.page_content))
        token_counts[page_num] = token_count
    
    for page, count in sorted(token_counts.items()):
        print(f"Page {page}: {count} tokens")
        
    return token_counts

# 使用方法
token_counts = count_tokens_per_page_transformers(documents,embedding_model)

Page 0: 462 tokens
Page 1: 408 tokens
Page 2: 381 tokens
Page 3: 410 tokens
Page 4: 389 tokens
Page 5: 388 tokens
Page 6: 386 tokens
Page 7: 405 tokens
Page 8: 440 tokens
Page 9: 461 tokens


In [86]:
text_splitter = CharacterTextSplitter(chunk_size=450, chunk_overlap=50,separator='\n')
texts = text_splitter.split_documents(documents)
print(len(texts))
texts[:5]

20


[Document(metadata={'source': './proxmox_documents/PVE_Trouble_Shooting_Paged.pdf', 'file_path': './proxmox_documents/PVE_Trouble_Shooting_Paged.pdf', 'page': 0, 'total_pages': 10, 'format': 'PDF 1.5', 'title': '', 'author': 'python-docx', 'subject': '', 'keywords': '', 'creator': 'Microsoft® Word 2016', 'producer': 'Microsoft® Word 2016', 'creationDate': "D:20241202104224+08'00'", 'modDate': "D:20241202104224+08'00'", 'trapped': ''}, page_content='PVE 常見問題與 Trouble Shooting \n問題 1：無法啟動虛擬機器 (VM) \n問題類型：VM 啟動問題 \n問題描述：在 Web 介面中嘗試啟動虛擬機器時出現錯誤訊息，如 "Cannot access storage" \n或 "Unable to start VM"。 \n排解方式： \n1. 使用 REST API 檢查虛擬機器狀態：GET \n/api2/json/nodes/{node}/qemu/{vmid}/status/current \n2. 確認儲存設備是否正常運作：GET /api2/json/nodes/{node}/storage \n3. 若儲存設備異常，重新掛載或修復存儲。 \n4. 嘗試手動啟動 VM：POST /api2/json/nodes/{node}/qemu/{vmid}/status/start \n注意事項：請確認 VM 所需的資源可用，並檢查是否有防火牆或權限\n問題。 \n問題 2：無法建立快照 \n問題類型：快照問題'),
 Document(metadata={'source': './proxmox_documents/PVE_Trouble_Shooting_Paged.pdf', 'file_pat

In [87]:
index_path="./faiss/trouble_shooting_index"
embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model,
    model_kwargs={'device': 'cpu'}, # 如果有 GPU 可以改為 'cuda'
    encode_kwargs={'normalize_embeddings': True}
)
if not os.path.exists(index_path):
    # 首次建立：使用 from_documents
    print("FAISS index 不存在快取，正在建立index")
    vectorstore = FAISS.from_documents(texts, embeddings)
    vectorstore.save_local(index_path)
    print("FAISS index 建立完成")
else:
    # 已存在：直接載入
    vectorstore = FAISS.load_local(index_path, embeddings,allow_dangerous_deserialization=True)
    print("FAISS index 存在快取，載入成功")

FAISS index 不存在快取，正在建立index
FAISS index 建立完成


In [88]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [89]:
query = "伺服器資源使用率過高"
for i in retriever.invoke(query):
    print(i.page_content,end='\n\n')

問題 3：伺服器資源使用率過高 
問題類型：資源管理問題 
問題描述：伺服器的 CPU、記憶體或磁碟使用率達到極限，影響 VM 性能。 
排解方式： 
1. 使用 REST API 檢查資源使用情況：GET /api2/json/nodes/{node}/status 
2. 若記憶體過高，檢查是否有過多 VM 啟動，並釋放資源：POST 
/api2/json/nodes/{node}/qemu/{vmid}/status/stop 
3. 若磁碟使用率過高，清理不必要的檔案或備份。 
注意事項：考慮提升伺服器硬體規格，或進行資源負載平衡。 
問題 4：無法連接到 PVE Web 介面 
問題類型：網路或權限問題 
問題描述：無法透過瀏覽器存取 PVE Web 介面，可能是網路設定或服務異常。 
排解方式： 
1. 使用 REST API 檢查伺服器網路狀態：GET /api2/json/nodes/{node}/network

問題 5：無法升級 Proxmox VE 
問題類型：系統升級問題 
問題描述：在升級 PVE 時，出現套件依賴或版本不相容問題。 
排解方式： 
1. 檢查目前 PVE 版本和可用升級：GET /api2/json/version 
2. 更新套件列表：apt-get update 
3. 執行升級指令：apt-get dist-upgrade 
4. 重新啟動伺服器以應用更新。 
注意事項：升級前建議備份虛擬機器和重要資料。 
問題 6：虛擬機器磁碟掛載失敗 
問題類型：磁碟掛載問題 
問題描述：嘗試將磁碟掛載到虛擬機器時，出現 "Failed to mount disk" 的錯誤。 
排解方式： 
1. 確認磁碟是否已正確新增到存儲池：GET /api2/json/nodes/{node}/storage 
2. 檢查磁碟是否已被其他虛擬機器使用。

問題 13：無法新增儲存設備到 PVE 
問題類型：儲存配置問題 
問題描述：在 Web 介面或命令列中嘗試新增儲存設備時失敗。 
排解方式： 
1. 確認儲存設備是否已正確連接到伺服器。 
2. 使用命令檢查儲存設備狀態：lsblk 或 fdisk -l。 
3. 手動將儲存設備掛載到目錄並測試可用性。 
4. 在 Web 介面中重新新增儲存設備：POST /api2/json/n